In [56]:
import pandas as pd
import pickle
import tensorflow as tf

In [57]:
def read_csv():
    df = pd.read_csv("../../feature-engineering/data/output/output-pairwise.csv")
    return df 

def read_model():
    with open('../../ts-train-model/data/output/trained_lr_model.pkl', 'rb') as f:
        trained_model = pickle.load(f)
    return trained_model

def read_tensor_model(path="../../ts-train-model/data/output/trained_tensor_model"):
    return tf.keras.models.load_model(path)

In [58]:
df = read_csv()

In [59]:
## sklearn model

model = read_model()

probabilities = model.predict_proba(df[["first_name_similarity", 'last_name_similarity', "first_name_length_diff", "last_name_length_diff", "role_similarity", "context_similarity"]])

df['match_probability'] = probabilities[:, 1]
df

,entity_1_uid,entity_1_first_name,entity_1_last_name,entity_1_role,entity_1_context,entity_2_uid,entity_2_first_name,entity_2_last_name,entity_2_role,entity_2_context,first_name_similarity,last_name_similarity,role_similarity,first_name_length_diff,last_name_length_diff,context_similarity,match_probability
0,8257682566,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,6082939390,victoria,guidry,assisting officer,Mentioned as providing assistance to Officer D...,0.000000,0.000000,0.720588,8,0,0.525490,0.000002
1,8257682566,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,7858057238,carolyn,dalton,arresting officer,Mentioned as one of the officers who arrested ...,0.000000,1.000000,0.740573,7,0,0.691130,0.940062
2,8257682566,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,6922728269,terry,bean,booking officer,Mentioned as one of the officers who booked th...,0.000000,0.611111,0.702555,5,2,0.558255,0.079090
3,8257682566,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,3362754676,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,0.000000,1.000000,1.000000,0,0,0.835393,0.984428
4,6082939390,victoria,guidry,assisting officer,Mentioned as providing assistance to Officer D...,7858057238,carolyn,dalton,arresting officer,Mentioned as one of the officers who arrested ...,0.490079,0.000000,0.808403,1,0,0.508758,0.000019
5,6082939390,victoria,guidry,assisting officer,Mentioned as providing assistance to Officer D...,6922728269,terry,bean,booking officer,Mentioned as one of the officers who booked th...,0.550000,0.000000,0.702555,3,2,0.452902,0.000051
6,6082939390,victoria,guidry,assisting officer,Mentioned as providing assistance to Officer D...,3362754676,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,0.000000,0.000000,0.720588,8,0,0.630117,0.000002
7,7858057238,carolyn,dalton,arresting officer,Mentioned as one of the officers who arrested ...,6922728269,terry,bean,booking officer,Mentioned as one of the officers who booked th...,0.561905,0.611111,0.793464,2,2,0.611255,0.587067
8,7858057238,carolyn,dalton,arresting officer,Mentioned as one of the officers who arrested ...,3362754676,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,0.000000,1.000000,0.740573,7,0,0.619506,0.929536
9,6922728269,terry,bean,booking officer,Mentioned as one of the officers who booked th...,3362754676,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,0.000000,0.611111,0.702555,5,2,0.607873,0.088274


In [60]:
### tensorflow model 

def scale_additional_features(additional_features, first_name_weight=1.0, last_name_weight=2.0):
    additional_features['first_name_similarity'] *= first_name_weight
    additional_features['last_name_similarity'] *= last_name_weight
    return additional_features


features = df[["first_name_length_diff", "last_name_length_diff", "role_similarity", "context_similarity"]]

additional_features = df[["first_name_similarity", "last_name_similarity"]]

additional_features_scaled = scale_additional_features(additional_features.copy())

X = [features, additional_features_scaled.iloc[:, 0], additional_features_scaled.iloc[:, 1]]

model = read_tensor_model()
probabilities = model.predict(X)

df['match_probability'] = probabilities.flatten()

df

1/1 [==============================] - 0s 20ms/step


,entity_1_uid,entity_1_first_name,entity_1_last_name,entity_1_role,entity_1_context,entity_2_uid,entity_2_first_name,entity_2_last_name,entity_2_role,entity_2_context,first_name_similarity,last_name_similarity,role_similarity,first_name_length_diff,last_name_length_diff,context_similarity,match_probability
0,8257682566,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,6082939390,victoria,guidry,assisting officer,Mentioned as providing assistance to Officer D...,0.000000,0.000000,0.720588,8,0,0.525490,5.301655e-21
1,8257682566,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,7858057238,carolyn,dalton,arresting officer,Mentioned as one of the officers who arrested ...,0.000000,1.000000,0.740573,7,0,0.691130,2.591180e-16
2,8257682566,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,6922728269,terry,bean,booking officer,Mentioned as one of the officers who booked th...,0.000000,0.611111,0.702555,5,2,0.558255,7.436328e-17
3,8257682566,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,3362754676,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,0.000000,1.000000,1.000000,0,0,0.835393,9.300182e-01
4,6082939390,victoria,guidry,assisting officer,Mentioned as providing assistance to Officer D...,7858057238,carolyn,dalton,arresting officer,Mentioned as one of the officers who arrested ...,0.490079,0.000000,0.808403,1,0,0.508758,2.160686e-05
5,6082939390,victoria,guidry,assisting officer,Mentioned as providing assistance to Officer D...,6922728269,terry,bean,booking officer,Mentioned as one of the officers who booked th...,0.550000,0.000000,0.702555,3,2,0.452902,2.430522e-14
6,6082939390,victoria,guidry,assisting officer,Mentioned as providing assistance to Officer D...,3362754676,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,0.000000,0.000000,0.720588,8,0,0.630117,4.513656e-21
7,7858057238,carolyn,dalton,arresting officer,Mentioned as one of the officers who arrested ...,6922728269,terry,bean,booking officer,Mentioned as one of the officers who booked th...,0.561905,0.611111,0.793464,2,2,0.611255,1.603397e-10
8,7858057238,carolyn,dalton,arresting officer,Mentioned as one of the officers who arrested ...,3362754676,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,0.000000,1.000000,0.740573,7,0,0.619506,2.542051e-16
9,6922728269,terry,bean,booking officer,Mentioned as one of the officers who booked th...,3362754676,NaN,dalton,verifying officer,Mentioned as one of the officers who verified ...,0.000000,0.611111,0.702555,5,2,0.607873,7.062901e-17


In [61]:
df.match_probability.value_counts()

match_probability
5.301655e-21    1
2.591180e-16    1
7.436328e-17    1
9.300182e-01    1
2.160686e-05    1
2.430522e-14    1
4.513656e-21    1
1.603397e-10    1
2.542051e-16    1
7.062901e-17    1
Name: count, dtype: int64